In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import sys
oldsysstdout = sys.stdout
class flushfile():
    def __init__(self, f):
        self.f = f
    def __getattr__(self,name): 
        return object.__getattribute__(self.f, name)
    def write(self, x):
        self.f.write(x)
        self.f.flush()
    def flush(self):
        self.f.flush()
sys.stdout = flushfile(sys.stdout)

In [3]:
import json

In [4]:
with open('../train.json', 'r') as f:
    train = json.load(f)
with open('../test.json', 'r') as f:
    test = json.load(f)

In [5]:
vec = []
for i in train:
    [vec.append(j) for j in i['ingredients']]

In [7]:
cats = np.unique((np.array(vec)))

In [8]:
from scipy.sparse import csr_matrix

In [9]:
X = csr_matrix(map(lambda y: np.array(map(lambda x: 1 if x in y['ingredients'] else 0, cats), dtype='int8'), train))

In [10]:
X.shape

(39774, 6714)

In [11]:
target = map(lambda x: x['cuisine'], train)

In [12]:
b, y = np.unique(target, return_inverse=True)

In [13]:
test_sp = csr_matrix(map(lambda y: np.array(map(lambda x: 1 if x in y['ingredients'] else 0, cats), dtype='int8'), test))

In [15]:
from sklearn.cross_validation import train_test_split, StratifiedKFold

In [17]:
cv = StratifiedKFold(y, n_folds = 10, random_state = 0)

In [18]:
from copy import copy

In [22]:
X_t = X
y_t = y

In [24]:
import lasagne
from lasagne.layers import InputLayer, DenseLayer, DropoutLayer
from lasagne.utils import floatX
import theano.tensor as T
import theano

Using gpu device 0: GeForce GTX 670 (CNMeM is disabled)


In [25]:
input_var = T.tensor3('inputs')
target_var = T.ivector('targets')

In [26]:
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(inputs.shape[0])
        np.random.shuffle(indices)
    for start_idx in range(0, inputs.shape[0] - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt].reshape((-1, 1, 6714)), targets[excerpt]

In [27]:
import time

In [ ]:
f = 1
accuracies = []
for train_index, test_index in cv:
    
    X_train = X_t[train_index]
    X_test = X_t[test_index]
    y_train = y_t[train_index]
    y_test = y_t[test_index]
    X_train = X_train.toarray()
    X_test = X_test.toarray()
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    y_train = y_train.astype('int32')
    y_test = y_test.astype('int32')
    
    net = {}
    net['input'] = InputLayer((None, 1, 6714),input_var=input_var)
    net['fc1'] = DenseLayer(net['input'], num_units=4000,
                            nonlinearity = lasagne.nonlinearities.rectify,
                            W = lasagne.init.GlorotUniform(gain = 'relu'))
    net['drop1'] = DropoutLayer(net['fc1'], p=0.5)
    net['fc2'] = DenseLayer(net['drop1'], num_units=2000,
                            nonlinearity = lasagne.nonlinearities.rectify,
                            W = lasagne.init.GlorotUniform(gain = 'relu'))
    net['drop2'] = DropoutLayer(net['fc2'], p=0.5)
    net['fc3'] = DenseLayer(net['drop2'], num_units=700,
                            nonlinearity = lasagne.nonlinearities.rectify,
                            W = lasagne.init.GlorotUniform(gain = 'relu'))
    net['drop3'] = DropoutLayer(net['fc3'], p=0.5)
    net['fc4'] = DenseLayer(net['drop3'], num_units=200,
                            nonlinearity = lasagne.nonlinearities.rectify,
                            W = lasagne.init.GlorotUniform(gain = 'relu'))
    net['drop4'] = DropoutLayer(net['fc4'], p=0.5)
    net['fc5'] = DenseLayer(net['drop4'], num_units=20, nonlinearity=lasagne.nonlinearities.softmax)
    output_layer = net['fc5']
    
    prediction = lasagne.layers.get_output(output_layer)
    loss = lasagne.objectives.categorical_crossentropy(prediction, target_var)
    loss = loss.mean()

    params = lasagne.layers.get_all_params(output_layer, trainable=True)
    updates = lasagne.updates.nesterov_momentum(
            loss, params, learning_rate=0.05, momentum=0.9)

    test_prediction = lasagne.layers.get_output(output_layer, deterministic=True)
    test_loss = lasagne.objectives.categorical_crossentropy(test_prediction,
                                                            target_var)
    test_loss = test_loss.mean()
    test_acc = T.mean(T.eq(T.argmax(test_prediction, axis=1), target_var),
                      dtype=theano.config.floatX)

    train_fn = theano.function([input_var, target_var], loss, updates=updates)

    val_fn = theano.function([input_var, target_var], [test_loss, test_acc])

    test_fn = theano.function([input_var], test_prediction)
    
    num_epochs = 1000
    batchsize = 2500
    best_epoch = 0
    best_acc = 0
    best_out = np.zeros((len(X_test), 20))

    print("Starting training... fold {:} of {:}".format(f, 10))
    f += 1
    for epoch in range(num_epochs):
        train_err = 0
        train_batches = 0
        start_time = time.time()
        for batch in iterate_minibatches(X_train, y_train, batchsize, shuffle=True):
            inputs, targets = batch
            train_err += train_fn(inputs, targets)
            train_batches += 1
        
        val_err = 0
        val_acc = 0
        val_batches = 0
        for batch in iterate_minibatches(X_test, y_test, 1, shuffle=False):
            inputs, targets = batch
            err, acc = val_fn(inputs, targets)
            val_err += err
            val_acc += acc
            val_batches += 1
        
        e_acc = val_acc / float(val_batches)
        if e_acc >= best_acc + .001:
            best_acc = copy(e_acc)
            best_epoch = epoch
        
        if epoch >= best_epoch + 10:
            print("Stopping at {} epoch with accuracy {}\nBest accuracy {} at {}".format(epoch, 
                                                                                         e_acc, best_acc, best_epoch))
            accuracies.append(best_acc)
            break
            
        print("Epoch {} of {} took {:.3f}s".format(epoch + 1, num_epochs, time.time() - start_time))
        print("  training loss:\t\t{:.6f}".format(train_err / train_batches))
        #print("  training accuracy:\t\t{:.6f}".format(train_acc / train_batches))
        print("  validation loss:\t\t{:.6f}".format(val_err / val_batches))
        print("  validation accuracy:\t\t{:.6f}".format(val_acc / val_batches))
        
print np.mean(accuracies), np.std(accuracies)

In [ ]:
f = 1
accuracies = []
for train_index, test_index in cv:
    
    X_train = X_t[train_index]
    X_test = X_t[test_index]
    y_train = y_t[train_index]
    y_test = y_t[test_index]
    X_train = X_train.toarray()
    X_test = X_test.toarray()
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    y_train = y_train.astype('int32')
    y_test = y_test.astype('int32')
    
    net = {}
    net['input'] = InputLayer((None, 1, 6714),input_var=input_var)
    net['fc1'] = DenseLayer(net['input'], num_units=4000,
                            nonlinearity = lasagne.nonlinearities.rectify,
                            W = lasagne.init.GlorotUniform(gain = 'relu'))
    net['fc2'] = DenseLayer(net['fc1'], num_units=2000,
                            nonlinearity = lasagne.nonlinearities.rectify,
                            W = lasagne.init.GlorotUniform(gain = 'relu'))
    net['fc3'] = DenseLayer(net['fc2'], num_units=700,
                            nonlinearity = lasagne.nonlinearities.rectify,
                            W = lasagne.init.GlorotUniform(gain = 'relu'))
    net['fc4'] = DenseLayer(net['fc3'], num_units=200,
                            nonlinearity = lasagne.nonlinearities.rectify,
                            W = lasagne.init.GlorotUniform(gain = 'relu'))
    net['fc5'] = DenseLayer(net['fc4'], num_units=20, nonlinearity=lasagne.nonlinearities.softmax)
    output_layer = net['fc5']
    
    prediction = lasagne.layers.get_output(output_layer)
    loss = lasagne.objectives.categorical_crossentropy(prediction, target_var)
    loss = loss.mean()

    params = lasagne.layers.get_all_params(output_layer, trainable=True)
    updates = lasagne.updates.nesterov_momentum(
            loss, params, learning_rate=0.05, momentum=0.9)

    test_prediction = lasagne.layers.get_output(output_layer, deterministic=True)
    test_loss = lasagne.objectives.categorical_crossentropy(test_prediction,
                                                            target_var)
    test_loss = test_loss.mean()
    test_acc = T.mean(T.eq(T.argmax(test_prediction, axis=1), target_var),
                      dtype=theano.config.floatX)

    train_fn = theano.function([input_var, target_var], loss, updates=updates)

    val_fn = theano.function([input_var, target_var], [test_loss, test_acc])

    test_fn = theano.function([input_var], test_prediction)
    
    num_epochs = 1000
    batchsize = 2500
    best_epoch = 0
    best_acc = 0
    best_out = np.zeros((len(X_test), 20))

    print("Starting training... fold {:} of {:}".format(f, 10))
    f += 1
    for epoch in range(num_epochs):
        train_err = 0
        train_batches = 0
        start_time = time.time()
        for batch in iterate_minibatches(X_train, y_train, batchsize, shuffle=True):
            inputs, targets = batch
            train_err += train_fn(inputs, targets)
            train_batches += 1
        
        val_err = 0
        val_acc = 0
        val_batches = 0
        for batch in iterate_minibatches(X_test, y_test, 1, shuffle=False):
            inputs, targets = batch
            err, acc = val_fn(inputs, targets)
            val_err += err
            val_acc += acc
            val_batches += 1
        
        e_acc = val_acc / float(val_batches)
        if e_acc >= best_acc + .001:
            best_acc = copy(e_acc)
            best_epoch = epoch
        
        if epoch >= best_epoch + 10:
            print("Stopping at {} epoch with accuracy {}\nBest accuracy {} at {}".format(epoch, 
                                                                                         e_acc, best_acc, best_epoch))
            accuracies.append(best_acc)
            break
            
        print("Epoch {} of {} took {:.3f}s".format(epoch + 1, num_epochs, time.time() - start_time))
        print("  training loss:\t\t{:.6f}".format(train_err / train_batches))
        #print("  training accuracy:\t\t{:.6f}".format(train_acc / train_batches))
        print("  validation loss:\t\t{:.6f}".format(val_err / val_batches))
        print("  validation accuracy:\t\t{:.6f}".format(val_acc / val_batches))
        
print np.mean(accuracies), np.std(accuracies)

In [ ]:
f = 1
accuracies = []
for train_index, test_index in cv:
    
    X_train = X_t[train_index]
    X_test = X_t[test_index]
    y_train = y_t[train_index]
    y_test = y_t[test_index]
    X_train = X_train.toarray()
    X_test = X_test.toarray()
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    y_train = y_train.astype('int32')
    y_test = y_test.astype('int32')
    
    net = {}
    net['input'] = InputLayer((None, 1, 6714),input_var=input_var)
    net['fc1'] = DenseLayer(net['input'], num_units=3000,
                            nonlinearity = lasagne.nonlinearities.rectify,
                            W = lasagne.init.GlorotUniform(gain = 'relu'))
    net['drop1'] = DropoutLayer(net['fc1'], p=0.5)
    net['fc4'] = DenseLayer(net['drop1'], num_units=200,
                            nonlinearity = lasagne.nonlinearities.rectify,
                            W = lasagne.init.GlorotUniform(gain = 'relu'))
    net['drop4'] = DropoutLayer(net['fc4'], p=0.5)
    net['fc5'] = DenseLayer(net['drop4'], num_units=20, nonlinearity=lasagne.nonlinearities.softmax)
    output_layer = net['fc5']
    
    prediction = lasagne.layers.get_output(output_layer)
    loss = lasagne.objectives.categorical_crossentropy(prediction, target_var)
    loss = loss.mean()

    params = lasagne.layers.get_all_params(output_layer, trainable=True)
    updates = lasagne.updates.nesterov_momentum(
            loss, params, learning_rate=0.05, momentum=0.9)

    test_prediction = lasagne.layers.get_output(output_layer, deterministic=True)
    test_loss = lasagne.objectives.categorical_crossentropy(test_prediction,
                                                            target_var)
    test_loss = test_loss.mean()
    test_acc = T.mean(T.eq(T.argmax(test_prediction, axis=1), target_var),
                      dtype=theano.config.floatX)

    train_fn = theano.function([input_var, target_var], loss, updates=updates)

    val_fn = theano.function([input_var, target_var], [test_loss, test_acc])

    test_fn = theano.function([input_var], test_prediction)
    
    num_epochs = 1000
    batchsize = 2500
    best_epoch = 0
    best_acc = 0
    best_out = np.zeros((len(X_test), 20))

    print("Starting training... fold {:} of {:}".format(f, 10))
    f += 1
    for epoch in range(num_epochs):
        train_err = 0
        train_batches = 0
        start_time = time.time()
        for batch in iterate_minibatches(X_train, y_train, batchsize, shuffle=True):
            inputs, targets = batch
            train_err += train_fn(inputs, targets)
            train_batches += 1
        
        val_err = 0
        val_acc = 0
        val_batches = 0
        for batch in iterate_minibatches(X_test, y_test, 1, shuffle=False):
            inputs, targets = batch
            err, acc = val_fn(inputs, targets)
            val_err += err
            val_acc += acc
            val_batches += 1
        
        e_acc = val_acc / float(val_batches)
        if e_acc >= best_acc + .001:
            best_acc = copy(e_acc)
            best_epoch = epoch
        
        if epoch >= best_epoch + 10:
            print("Stopping at {} epoch with accuracy {}\nBest accuracy {} at {}".format(epoch, 
                                                                                         e_acc, best_acc, best_epoch))
            accuracies.append(best_acc)
            break
            
        print("Epoch {} of {} took {:.3f}s".format(epoch + 1, num_epochs, time.time() - start_time))
        print("  training loss:\t\t{:.6f}".format(train_err / train_batches))
        #print("  training accuracy:\t\t{:.6f}".format(train_acc / train_batches))
        print("  validation loss:\t\t{:.6f}".format(val_err / val_batches))
        print("  validation accuracy:\t\t{:.6f}".format(val_acc / val_batches))
        
print np.mean(accuracies), np.std(accuracies)

In [ ]:
f = 1
accuracies = []
for train_index, test_index in cv:
    
    X_train = X_t[train_index]
    X_test = X_t[test_index]
    y_train = y_t[train_index]
    y_test = y_t[test_index]
    X_train = X_train.toarray()
    X_test = X_test.toarray()
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    y_train = y_train.astype('int32')
    y_test = y_test.astype('int32')
    
    net = {}
    net['input'] = InputLayer((None, 1, 6714),input_var=input_var)
    net['fc1'] = DenseLayer(net['input'], num_units=3000,
                            nonlinearity = lasagne.nonlinearities.rectify,
                            W = lasagne.init.GlorotUniform(gain = 'relu'))
    net['fc4'] = DenseLayer(net['fc1'], num_units=200,
                            nonlinearity = lasagne.nonlinearities.rectify,
                            W = lasagne.init.GlorotUniform(gain = 'relu'))
    net['fc5'] = DenseLayer(net['fc4'], num_units=20, nonlinearity=lasagne.nonlinearities.softmax)
    output_layer = net['fc5']
    
    prediction = lasagne.layers.get_output(output_layer)
    loss = lasagne.objectives.categorical_crossentropy(prediction, target_var)
    loss = loss.mean()

    params = lasagne.layers.get_all_params(output_layer, trainable=True)
    updates = lasagne.updates.nesterov_momentum(
            loss, params, learning_rate=0.05, momentum=0.9)

    test_prediction = lasagne.layers.get_output(output_layer, deterministic=True)
    test_loss = lasagne.objectives.categorical_crossentropy(test_prediction,
                                                            target_var)
    test_loss = test_loss.mean()
    test_acc = T.mean(T.eq(T.argmax(test_prediction, axis=1), target_var),
                      dtype=theano.config.floatX)

    train_fn = theano.function([input_var, target_var], loss, updates=updates)

    val_fn = theano.function([input_var, target_var], [test_loss, test_acc])

    test_fn = theano.function([input_var], test_prediction)
    
    num_epochs = 1000
    batchsize = 2500
    best_epoch = 0
    best_acc = 0
    best_out = np.zeros((len(X_test), 20))

    print("Starting training... fold {:} of {:}".format(f, 10))
    f += 1
    for epoch in range(num_epochs):
        train_err = 0
        train_batches = 0
        start_time = time.time()
        for batch in iterate_minibatches(X_train, y_train, batchsize, shuffle=True):
            inputs, targets = batch
            train_err += train_fn(inputs, targets)
            train_batches += 1
        
        val_err = 0
        val_acc = 0
        val_batches = 0
        for batch in iterate_minibatches(X_test, y_test, 1, shuffle=False):
            inputs, targets = batch
            err, acc = val_fn(inputs, targets)
            val_err += err
            val_acc += acc
            val_batches += 1
        
        e_acc = val_acc / float(val_batches)
        if e_acc >= best_acc + .001:
            best_acc = copy(e_acc)
            best_epoch = epoch
        
        if epoch >= best_epoch + 10:
            print("Stopping at {} epoch with accuracy {}\nBest accuracy {} at {}".format(epoch, 
                                                                                         e_acc, best_acc, best_epoch))
            accuracies.append(best_acc)
            break
            
        print("Epoch {} of {} took {:.3f}s".format(epoch + 1, num_epochs, time.time() - start_time))
        print("  training loss:\t\t{:.6f}".format(train_err / train_batches))
        #print("  training accuracy:\t\t{:.6f}".format(train_acc / train_batches))
        print("  validation loss:\t\t{:.6f}".format(val_err / val_batches))
        print("  validation accuracy:\t\t{:.6f}".format(val_acc / val_batches))
        
print np.mean(accuracies), np.std(accuracies)

In [ ]:
f = 1
accuracies = []
for train_index, test_index in cv:
    
    X_train = X_t[train_index]
    X_test = X_t[test_index]
    y_train = y_t[train_index]
    y_test = y_t[test_index]
    X_train = X_train.toarray()
    X_test = X_test.toarray()
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    y_train = y_train.astype('int32')
    y_test = y_test.astype('int32')
    
    net = {}
    net['input'] = InputLayer((None, 1, 6714),input_var=input_var)
    net['fc1'] = DenseLayer(net['input'], num_units=3000,
                            nonlinearity = lasagne.nonlinearities.rectify,
                            W = lasagne.init.GlorotUniform(gain = 'relu'))
    net['fc5'] = DenseLayer(net['fc1'], num_units=20, nonlinearity=lasagne.nonlinearities.softmax)
    output_layer = net['fc5']
    
    prediction = lasagne.layers.get_output(output_layer)
    loss = lasagne.objectives.categorical_crossentropy(prediction, target_var)
    loss = loss.mean()

    params = lasagne.layers.get_all_params(output_layer, trainable=True)
    updates = lasagne.updates.nesterov_momentum(
            loss, params, learning_rate=0.05, momentum=0.9)

    test_prediction = lasagne.layers.get_output(output_layer, deterministic=True)
    test_loss = lasagne.objectives.categorical_crossentropy(test_prediction,
                                                            target_var)
    test_loss = test_loss.mean()
    test_acc = T.mean(T.eq(T.argmax(test_prediction, axis=1), target_var),
                      dtype=theano.config.floatX)

    train_fn = theano.function([input_var, target_var], loss, updates=updates)

    val_fn = theano.function([input_var, target_var], [test_loss, test_acc])

    test_fn = theano.function([input_var], test_prediction)
    
    num_epochs = 1000
    batchsize = 2500
    best_epoch = 0
    best_acc = 0
    best_out = np.zeros((len(X_test), 20))

    print("Starting training... fold {:} of {:}".format(f, 10))
    f += 1
    for epoch in range(num_epochs):
        train_err = 0
        train_batches = 0
        start_time = time.time()
        for batch in iterate_minibatches(X_train, y_train, batchsize, shuffle=True):
            inputs, targets = batch
            train_err += train_fn(inputs, targets)
            train_batches += 1
        
        val_err = 0
        val_acc = 0
        val_batches = 0
        for batch in iterate_minibatches(X_test, y_test, 1, shuffle=False):
            inputs, targets = batch
            err, acc = val_fn(inputs, targets)
            val_err += err
            val_acc += acc
            val_batches += 1
        
        e_acc = val_acc / float(val_batches)
        if e_acc >= best_acc + .001:
            best_acc = copy(e_acc)
            best_epoch = epoch
        
        if epoch >= best_epoch + 10:
            print("Stopping at {} epoch with accuracy {}\nBest accuracy {} at {}".format(epoch, 
                                                                                         e_acc, best_acc, best_epoch))
            accuracies.append(best_acc)
            break
            
        print("Epoch {} of {} took {:.3f}s".format(epoch + 1, num_epochs, time.time() - start_time))
        print("  training loss:\t\t{:.6f}".format(train_err / train_batches))
        #print("  training accuracy:\t\t{:.6f}".format(train_acc / train_batches))
        print("  validation loss:\t\t{:.6f}".format(val_err / val_batches))
        print("  validation accuracy:\t\t{:.6f}".format(val_acc / val_batches))
        
print np.mean(accuracies), np.std(accuracies)

Starting training... fold 1 of 10
Epoch 1 of 1000 took 14.409s
  training loss:		2.765840
  validation loss:		2.539110
  validation accuracy:		0.237390
Epoch 2 of 1000 took 14.427s
  training loss:		2.433120
  validation loss:		2.324845
  validation accuracy:		0.407026
Epoch 3 of 1000 took 14.473s
  training loss:		2.209782
  validation loss:		2.105381
  validation accuracy:		0.459975
Epoch 4 of 1000 took 14.427s
  training loss:		1.995313
  validation loss:		1.911451
  validation accuracy:		0.490339
Epoch 5 of 1000 took 14.559s
  training loss:		1.818339
  validation loss:		1.759284
  validation accuracy:		0.520452
Epoch 6 of 1000 took 14.429s
  training loss:		1.680330
  validation loss:		1.638233
  validation accuracy:		0.547553
Epoch 7 of 1000 took 14.402s
  training loss:		1.566472
  validation loss:		1.537556
  validation accuracy:		0.568381
Epoch 8 of 1000 took 14.423s
  training loss:		1.473575
  validation loss:		1.453468
  validation accuracy:		0.590715
Epoch 9 of 1000 took 1

In [ ]:
f = 1
accuracies = []
for train_index, test_index in cv:
    
    X_train = X_t[train_index]
    X_test = X_t[test_index]
    y_train = y_t[train_index]
    y_test = y_t[test_index]
    X_train = X_train.toarray()
    X_test = X_test.toarray()
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    y_train = y_train.astype('int32')
    y_test = y_test.astype('int32')
    
    net = {}
    net['input'] = InputLayer((None, 1, 6714),input_var=input_var)
    net['fc1'] = DenseLayer(net['input'], num_units=1000,
                            nonlinearity = lasagne.nonlinearities.rectify,
                            W = lasagne.init.GlorotUniform(gain = 'relu'))
    net['fc5'] = DenseLayer(net['fc1'], num_units=20, nonlinearity=lasagne.nonlinearities.softmax)
    output_layer = net['fc5']
    
    prediction = lasagne.layers.get_output(output_layer)
    loss = lasagne.objectives.categorical_crossentropy(prediction, target_var)
    loss = loss.mean()

    params = lasagne.layers.get_all_params(output_layer, trainable=True)
    updates = lasagne.updates.nesterov_momentum(
            loss, params, learning_rate=0.05, momentum=0.9)

    test_prediction = lasagne.layers.get_output(output_layer, deterministic=True)
    test_loss = lasagne.objectives.categorical_crossentropy(test_prediction,
                                                            target_var)
    test_loss = test_loss.mean()
    test_acc = T.mean(T.eq(T.argmax(test_prediction, axis=1), target_var),
                      dtype=theano.config.floatX)

    train_fn = theano.function([input_var, target_var], loss, updates=updates)

    val_fn = theano.function([input_var, target_var], [test_loss, test_acc])

    test_fn = theano.function([input_var], test_prediction)
    
    num_epochs = 1000
    batchsize = 2500
    best_epoch = 0
    best_acc = 0
    best_out = np.zeros((len(X_test), 20))

    print("Starting training... fold {:} of {:}".format(f, 10))
    f += 1
    for epoch in range(num_epochs):
        train_err = 0
        train_batches = 0
        start_time = time.time()
        for batch in iterate_minibatches(X_train, y_train, batchsize, shuffle=True):
            inputs, targets = batch
            train_err += train_fn(inputs, targets)
            train_batches += 1
        
        val_err = 0
        val_acc = 0
        val_batches = 0
        for batch in iterate_minibatches(X_test, y_test, 1, shuffle=False):
            inputs, targets = batch
            err, acc = val_fn(inputs, targets)
            val_err += err
            val_acc += acc
            val_batches += 1
        
        e_acc = val_acc / float(val_batches)
        if e_acc >= best_acc + .001:
            best_acc = copy(e_acc)
            best_epoch = epoch
        
        if epoch >= best_epoch + 10:
            print("Stopping at {} epoch with accuracy {}\nBest accuracy {} at {}".format(epoch, 
                                                                                         e_acc, best_acc, best_epoch))
            accuracies.append(best_acc)
            break
            
        print("Epoch {} of {} took {:.3f}s".format(epoch + 1, num_epochs, time.time() - start_time))
        print("  training loss:\t\t{:.6f}".format(train_err / train_batches))
        #print("  training accuracy:\t\t{:.6f}".format(train_acc / train_batches))
        print("  validation loss:\t\t{:.6f}".format(val_err / val_batches))
        print("  validation accuracy:\t\t{:.6f}".format(val_acc / val_batches))
        
print np.mean(accuracies), np.std(accuracies)

Starting training... fold 1 of 10
Epoch 1 of 1000 took 5.806s
  training loss:		2.816404
  validation loss:		2.579345
  validation accuracy:		0.255709
Epoch 2 of 1000 took 5.792s
  training loss:		2.466186
  validation loss:		2.355771
  validation accuracy:		0.364366
Epoch 3 of 1000 took 5.983s
  training loss:		2.239469
  validation loss:		2.133723
  validation accuracy:		0.454956
Epoch 4 of 1000 took 5.949s
  training loss:		2.022797
  validation loss:		1.932030
  validation accuracy:		0.484567
Epoch 5 of 1000 took 5.802s
  training loss:		1.838574
  validation loss:		1.775044
  validation accuracy:		0.515684
Epoch 6 of 1000 took 5.842s
  training loss:		1.697697
  validation loss:		1.651167
  validation accuracy:		0.540276
Epoch 7 of 1000 took 6.045s
  training loss:		1.583089
  validation loss:		1.550185
  validation accuracy:		0.563864
Epoch 8 of 1000 took 6.158s
  training loss:		1.489510
  validation loss:		1.466691
  validation accuracy:		0.586951
Epoch 9 of 1000 took 5.845s
  

In [38]:
f = 1
accuracies = []
for train_index, test_index in cv:
    
    X_train = X_t[train_index]
    X_test = X_t[test_index]
    y_train = y_t[train_index]
    y_test = y_t[test_index]
    X_train = X_train.toarray()
    X_test = X_test.toarray()
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    y_train = y_train.astype('int32')
    y_test = y_test.astype('int32')
    
    net = {}
    net['input'] = InputLayer((None, 1, 6714),input_var=input_var)
    net['fc1'] = DenseLayer(net['input'], num_units=512,
                            nonlinearity = lasagne.nonlinearities.rectify,
                            W = lasagne.init.GlorotUniform(gain = 'relu'))
    net['fc5'] = DenseLayer(net['fc1'], num_units=20, nonlinearity=lasagne.nonlinearities.softmax)
    output_layer = net['fc5']
    
    prediction = lasagne.layers.get_output(output_layer)
    loss = lasagne.objectives.categorical_crossentropy(prediction, target_var)
    loss = loss.mean()

    params = lasagne.layers.get_all_params(output_layer, trainable=True)
    updates = lasagne.updates.nesterov_momentum(
            loss, params, learning_rate=0.05, momentum=0.9)

    test_prediction = lasagne.layers.get_output(output_layer, deterministic=True)
    test_loss = lasagne.objectives.categorical_crossentropy(test_prediction,
                                                            target_var)
    test_loss = test_loss.mean()
    test_acc = T.mean(T.eq(T.argmax(test_prediction, axis=1), target_var),
                      dtype=theano.config.floatX)

    train_fn = theano.function([input_var, target_var], loss, updates=updates)

    val_fn = theano.function([input_var, target_var], [test_loss, test_acc])

    test_fn = theano.function([input_var], test_prediction)
    
    num_epochs = 1000
    batchsize = 2500
    best_epoch = 0
    best_acc = 0
    best_out = np.zeros((len(X_test), 20))

    print("Starting training... fold {:} of {:}".format(f, 10))
    f += 1
    for epoch in range(num_epochs):
        train_err = 0
        train_batches = 0
        start_time = time.time()
        for batch in iterate_minibatches(X_train, y_train, batchsize, shuffle=True):
            inputs, targets = batch
            train_err += train_fn(inputs, targets)
            train_batches += 1
        
        val_err = 0
        val_acc = 0
        val_batches = 0
        for batch in iterate_minibatches(X_test, y_test, 1, shuffle=False):
            inputs, targets = batch
            err, acc = val_fn(inputs, targets)
            val_err += err
            val_acc += acc
            val_batches += 1
        
        e_acc = val_acc / float(val_batches)
        if e_acc >= best_acc + .001:
            best_acc = copy(e_acc)
            best_epoch = epoch
        
        if epoch >= best_epoch + 10:
            print("Stopping at {} epoch with accuracy {}\nBest accuracy {} at {}".format(epoch, 
                                                                                         e_acc, best_acc, best_epoch))
            accuracies.append(best_acc)
            break
            
        print("Epoch {} of {} took {:.3f}s".format(epoch + 1, num_epochs, time.time() - start_time))
        print("  training loss:\t\t{:.6f}".format(train_err / train_batches))
        #print("  training accuracy:\t\t{:.6f}".format(train_acc / train_batches))
        print("  validation loss:\t\t{:.6f}".format(val_err / val_batches))
        print("  validation accuracy:\t\t{:.6f}".format(val_acc / val_batches))
        
print np.mean(accuracies), np.std(accuracies)

Starting training... fold 1 of 10
Epoch 1 of 1000 took 4.078s
  training loss:		2.842520
  validation loss:		2.611527
  validation accuracy:		0.236136
Epoch 2 of 1000 took 3.897s
  training loss:		2.483814
  validation loss:		2.371587
  validation accuracy:		0.347302
Epoch 3 of 1000 took 4.008s
  training loss:		2.250537
  validation loss:		2.135278
  validation accuracy:		0.460477
Epoch 4 of 1000 took 3.870s
  training loss:		2.017525
  validation loss:		1.926483
  validation accuracy:		0.483061
Epoch 5 of 1000 took 4.248s
  training loss:		1.834082
  validation loss:		1.770382
  validation accuracy:		0.514178
Epoch 6 of 1000 took 3.841s
  training loss:		1.691702
  validation loss:		1.648450
  validation accuracy:		0.542535
Epoch 7 of 1000 took 4.014s
  training loss:		1.579259
  validation loss:		1.548727
  validation accuracy:		0.566625
Epoch 8 of 1000 took 4.044s
  training loss:		1.485451
  validation loss:		1.465377
  validation accuracy:		0.589962
Epoch 9 of 1000 took 4.095s
  

In [ ]:
f = 1
accuracies = []
for train_index, test_index in cv:
    
    X_train = X_t[train_index]
    X_test = X_t[test_index]
    y_train = y_t[train_index]
    y_test = y_t[test_index]
    X_train = X_train.toarray()
    X_test = X_test.toarray()
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    y_train = y_train.astype('int32')
    y_test = y_test.astype('int32')
    
    net = {}
    net['input'] = InputLayer((None, 1, 6714),input_var=input_var)
    net['fc1'] = DenseLayer(net['input'], num_units=4000,
                            nonlinearity = lasagne.nonlinearities.rectify,
                            W = lasagne.init.GlorotUniform(gain = 'relu'))
    net['drop1'] = DropoutLayer(net['fc1'], p=0.75)
    net['fc2'] = DenseLayer(net['drop1'], num_units=2000,
                            nonlinearity = lasagne.nonlinearities.rectify,
                            W = lasagne.init.GlorotUniform(gain = 'relu'))
    net['drop2'] = DropoutLayer(net['fc2'], p=0.75)
    net['fc3'] = DenseLayer(net['drop2'], num_units=700,
                            nonlinearity = lasagne.nonlinearities.rectify,
                            W = lasagne.init.GlorotUniform(gain = 'relu'))
    net['drop3'] = DropoutLayer(net['fc3'], p=0.75)
    net['fc4'] = DenseLayer(net['drop3'], num_units=200,
                            nonlinearity = lasagne.nonlinearities.rectify,
                            W = lasagne.init.GlorotUniform(gain = 'relu'))
    net['drop4'] = DropoutLayer(net['fc4'], p=0.75)
    net['fc5'] = DenseLayer(net['drop4'], num_units=20, nonlinearity=lasagne.nonlinearities.softmax)
    output_layer = net['fc5']
    
    prediction = lasagne.layers.get_output(output_layer)
    loss = lasagne.objectives.categorical_crossentropy(prediction, target_var)
    loss = loss.mean()

    params = lasagne.layers.get_all_params(output_layer, trainable=True)
    updates = lasagne.updates.nesterov_momentum(
            loss, params, learning_rate=0.05, momentum=0.9)

    test_prediction = lasagne.layers.get_output(output_layer, deterministic=True)
    test_loss = lasagne.objectives.categorical_crossentropy(test_prediction,
                                                            target_var)
    test_loss = test_loss.mean()
    test_acc = T.mean(T.eq(T.argmax(test_prediction, axis=1), target_var),
                      dtype=theano.config.floatX)

    train_fn = theano.function([input_var, target_var], loss, updates=updates)

    val_fn = theano.function([input_var, target_var], [test_loss, test_acc])

    test_fn = theano.function([input_var], test_prediction)
    
    num_epochs = 1000
    batchsize = 2500
    best_epoch = 0
    best_acc = 0
    best_out = np.zeros((len(X_test), 20))

    print("Starting training... fold {:} of {:}".format(f, 10))
    f += 1
    for epoch in range(num_epochs):
        train_err = 0
        train_batches = 0
        start_time = time.time()
        for batch in iterate_minibatches(X_train, y_train, batchsize, shuffle=True):
            inputs, targets = batch
            train_err += train_fn(inputs, targets)
            train_batches += 1
        
        val_err = 0
        val_acc = 0
        val_batches = 0
        for batch in iterate_minibatches(X_test, y_test, 1, shuffle=False):
            inputs, targets = batch
            err, acc = val_fn(inputs, targets)
            val_err += err
            val_acc += acc
            val_batches += 1
        
        e_acc = val_acc / float(val_batches)
        if e_acc >= best_acc + .001:
            best_acc = copy(e_acc)
            best_epoch = epoch
        
        if epoch >= best_epoch + 10:
            print("Stopping at {} epoch with accuracy {}\nBest accuracy {} at {}".format(epoch, 
                                                                                         e_acc, best_acc, best_epoch))
            accuracies.append(best_acc)
            break
            
        print("Epoch {} of {} took {:.3f}s".format(epoch + 1, num_epochs, time.time() - start_time))
        print("  training loss:\t\t{:.6f}".format(train_err / train_batches))
        #print("  training accuracy:\t\t{:.6f}".format(train_acc / train_batches))
        print("  validation loss:\t\t{:.6f}".format(val_err / val_batches))
        print("  validation accuracy:\t\t{:.6f}".format(val_acc / val_batches))
        
print np.mean(accuracies), np.std(accuracies)

In [ ]:
f = 1
accuracies = []
for train_index, test_index in cv:
    
    X_train = X_t[train_index]
    X_test = X_t[test_index]
    y_train = y_t[train_index]
    y_test = y_t[test_index]
    X_train = X_train.toarray()
    X_test = X_test.toarray()
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    y_train = y_train.astype('int32')
    y_test = y_test.astype('int32')
    
    net = {}
    net['input'] = InputLayer((None, 1, 6714),input_var=input_var)
    net['fc1'] = DenseLayer(net['input'], num_units=4000,
                            nonlinearity = lasagne.nonlinearities.rectify,
                            W = lasagne.init.GlorotUniform(gain = 'relu'))
    net['drop1'] = DropoutLayer(net['fc1'], p=0.25)
    net['fc2'] = DenseLayer(net['drop1'], num_units=2000,
                            nonlinearity = lasagne.nonlinearities.rectify,
                            W = lasagne.init.GlorotUniform(gain = 'relu'))
    net['drop2'] = DropoutLayer(net['fc2'], p=0.25)
    net['fc3'] = DenseLayer(net['drop2'], num_units=700,
                            nonlinearity = lasagne.nonlinearities.rectify,
                            W = lasagne.init.GlorotUniform(gain = 'relu'))
    net['drop3'] = DropoutLayer(net['fc3'], p=0.25)
    net['fc4'] = DenseLayer(net['drop3'], num_units=200,
                            nonlinearity = lasagne.nonlinearities.rectify,
                            W = lasagne.init.GlorotUniform(gain = 'relu'))
    net['drop4'] = DropoutLayer(net['fc4'], p=0.25)
    net['fc5'] = DenseLayer(net['drop4'], num_units=20, nonlinearity=lasagne.nonlinearities.softmax)
    output_layer = net['fc5']
    
    prediction = lasagne.layers.get_output(output_layer)
    loss = lasagne.objectives.categorical_crossentropy(prediction, target_var)
    loss = loss.mean()

    params = lasagne.layers.get_all_params(output_layer, trainable=True)
    updates = lasagne.updates.nesterov_momentum(
            loss, params, learning_rate=0.05, momentum=0.9)

    test_prediction = lasagne.layers.get_output(output_layer, deterministic=True)
    test_loss = lasagne.objectives.categorical_crossentropy(test_prediction,
                                                            target_var)
    test_loss = test_loss.mean()
    test_acc = T.mean(T.eq(T.argmax(test_prediction, axis=1), target_var),
                      dtype=theano.config.floatX)

    train_fn = theano.function([input_var, target_var], loss, updates=updates)

    val_fn = theano.function([input_var, target_var], [test_loss, test_acc])

    test_fn = theano.function([input_var], test_prediction)
    
    num_epochs = 1000
    batchsize = 2500
    best_epoch = 0
    best_acc = 0
    best_out = np.zeros((len(X_test), 20))

    print("Starting training... fold {:} of {:}".format(f, 10))
    f += 1
    for epoch in range(num_epochs):
        train_err = 0
        train_batches = 0
        start_time = time.time()
        for batch in iterate_minibatches(X_train, y_train, batchsize, shuffle=True):
            inputs, targets = batch
            train_err += train_fn(inputs, targets)
            train_batches += 1
        
        val_err = 0
        val_acc = 0
        val_batches = 0
        for batch in iterate_minibatches(X_test, y_test, 1, shuffle=False):
            inputs, targets = batch
            err, acc = val_fn(inputs, targets)
            val_err += err
            val_acc += acc
            val_batches += 1
        
        e_acc = val_acc / float(val_batches)
        if e_acc >= best_acc + .001:
            best_acc = copy(e_acc)
            best_epoch = epoch
        
        if epoch >= best_epoch + 10:
            print("Stopping at {} epoch with accuracy {}\nBest accuracy {} at {}".format(epoch, 
                                                                                         e_acc, best_acc, best_epoch))
            accuracies.append(best_acc)
            break
            
        print("Epoch {} of {} took {:.3f}s".format(epoch + 1, num_epochs, time.time() - start_time))
        print("  training loss:\t\t{:.6f}".format(train_err / train_batches))
        #print("  training accuracy:\t\t{:.6f}".format(train_acc / train_batches))
        print("  validation loss:\t\t{:.6f}".format(val_err / val_batches))
        print("  validation accuracy:\t\t{:.6f}".format(val_acc / val_batches))
        
print np.mean(accuracies), np.std(accuracies)